<h2 style="color:#1f77b4;">Live Coding Parte 1</h2>

In [ ]:
import sys
import numpy as np
import pandas as pd

print("Python:", sys.version.split()[0])
print("Ejecutable:", sys.executable)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)


<h5 style="color:#1f77b4;">1. Generar un dataset de ejemplo.</h5>

dataset “Ventas PyME”

In [ ]:
np.random.seed(42)

n = 30000
df_demo = pd.DataFrame({
    "fecha": pd.date_range("2025-01-01", periods=n, freq="D"),
    "sucursal": np.random.choice(["Centro", "Norte", "Sur"], size=n, p=[0.5, 0.3, 0.2]),
    "canal": np.random.choice(["Online", "Tienda", "Mayorista"], size=n, p=[0.4, 0.45, 0.15]),
    "categoria": np.random.choice(["Abarrotes", "Limpieza", "Bebidas"], size=n),
    "unidades": np.random.poisson(lam=12, size=n).astype(float),
    "precio_unitario": np.round(np.random.normal(loc=6500, scale=1200, size=n), 0).astype(float),
    "descuento_pct": np.round(np.clip(np.random.normal(loc=0.08, scale=0.06, size=n), 0, 0.35), 2),
})
# Ventas (monto) base
df_demo["ventas"] = np.round(df_demo["unidades"] * df_demo["precio_unitario"] * (1 - df_demo["descuento_pct"]), 0)

# 1) Falta descuento más seguido en canal Tienda (patrón realista)
mask_desc = (df_demo["canal"] == "Tienda") & (np.random.rand(n) < 0.35)
df_demo.loc[mask_desc, "descuento_pct"] = np.nan

# 2) Falta precio_unitario en algunos registros (errores de carga)
mask_precio = np.random.rand(n) < 0.12
df_demo.loc[mask_precio, "precio_unitario"] = np.nan

# 3) Falta canal en pocos casos (dato categórico)
mask_canal = np.random.rand(n) < 0.10
df_demo.loc[mask_canal, "canal"] = np.nan

# ---- OUTLIERS ----
# Un día con venta extremadamente alta (outlier)
df_demo.loc[np.random.choice(df_demo.index, 1), "ventas"] *= 8

# Precio unitario extremadamente alto (outlier)
df_demo.loc[np.random.choice(df_demo.index, 1), "precio_unitario"] *= 5

# Unidades extremadamente altas (outlier)
df_demo.loc[np.random.choice(df_demo.index, 1), "unidades"] *= 6

df_demo.head(10)


In [ ]:
df_demo.info()

<h5 style="color:#1f77b4;">2. Detectar valores perdidos con isnull() y contarlos.</h5>

In [ ]:
df_demo.shape

In [ ]:
df_demo.isnull().sum()

In [ ]:
nulos = df_demo.isnull().sum()
porcentaje = (df_demo.isnull().mean() * 100).round(2)

resumen_nulos = pd.DataFrame({
    "nulos": nulos,
    "porcentaje_%": porcentaje
}).sort_values("nulos", ascending=False)

resumen_nulos



In [ ]:
total_nulos = df_demo.isnull().sum().sum()
filas_con_nulos = df_demo.isnull().any(axis=1).sum()

print("Total de nulos en el dataset:", total_nulos)
print("Filas con al menos un nulo:", filas_con_nulos)


<h5 style="color:#1f77b4;">3. Visualizar datos faltantes con missingno y Seaborn.</h5>

Missingno: matriz de nulos<br>
* Negro/gris = dato presente
* Blanco = dato faltante

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

msno.matrix(df_demo, figsize=(8, 3), sparkline=False)
plt.title("Matriz de valores faltantes (df_demo)")
plt.show()


Seaborn: heatmap de isnull()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

msno.bar(df_demo, figsize=(8, 3))
plt.title("Conteo de datos presentes por columna")
plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
sns.heatmap(df_demo.isnull(), cbar=False)
plt.title("Mapa de calor de nulos (primeras filas)")
plt.show()


<h5 style="color:#1f77b4;">4. Eliminar filas o columnas con dropna().</h5>

In [ ]:
df_demo.isnull().sum()

Eliminar filas solo si faltan columnas críticas: no aceptamos registros sin precio_unitario ni canal.

In [ ]:
df_demo["canal"].unique()

In [ ]:
df_demo["canal"].value_counts()

In [ ]:
df_drop_criticos = df_demo.dropna(subset=["precio_unitario", "canal"])

print("Filas antes:", df_demo.shape[0])
print("Filas después:", df_drop_criticos.shape[0])


<h5 style="color:#1f77b4;">5. Imputar datos numéricos con media y mediana, y categóricos con moda o nueva categoría.</h5>

In [ ]:
df_demo.isnull().sum()

Imputación de datos numéricos<br>
a) Con la media

In [ ]:
df_media = df_demo.copy()

df_media["precio_unitario"] = df_media["precio_unitario"].fillna(df_media["precio_unitario"].mean())

df_media["descuento_pct"] = df_media["descuento_pct"].fillna(df_media["descuento_pct"].mean())

df_media.isnull().sum()


b) Con la mediana

In [ ]:
df_mediana = df_demo.copy()

df_mediana["precio_unitario"] = df_mediana["precio_unitario"].fillna(
    df_mediana["precio_unitario"].median()
)

df_mediana["descuento_pct"] = df_mediana["descuento_pct"].fillna(
    df_mediana["descuento_pct"].median()
)

df_mediana.isnull().sum()


2) Imputación de datos categóricos: Con la moda

In [ ]:
df_moda = df_demo.copy()

In [ ]:
moda_canal = df_moda["canal"].mode()[0]
moda_canal

In [ ]:
df_moda["canal"] = df_moda["canal"].fillna(moda_canal)

df_moda.isnull().sum()


<h5 style="color:#1f77b4;">6. Detectar outliers con IQR y Z-score.</h5>

Lo aplicamos a precio_unitario y ventas

Detección de outliers con IQR

In [ ]:
import numpy as np

def detectar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    limite_inf = Q1 - 1.5 * IQR
    limite_sup = Q3 + 1.5 * IQR

    outliers = df[
        (df[columna] < limite_inf) | (df[columna] > limite_sup)
    ]

    return outliers, limite_inf, limite_sup


In [ ]:
out_iqr_precio, li, ls = detectar_outliers_iqr(df_demo, "precio_unitario")

print("Outliers en precio_unitario (IQR):", out_iqr_precio.shape[0])


In [ ]:
out_iqr_precio.shape

In [ ]:
out_iqr_precio["precio_unitario"].describe()

Detección de outliers con Z-score

In [ ]:
from scipy.stats import zscore

df_z = df_demo.copy()

df_z["z_ventas"] = zscore(df_z["ventas"], nan_policy="omit")

outliers_z = df_z[abs(df_z["z_ventas"]) > 3]

print("Outliers en ventas (Z-score):", outliers_z.shape[0])


In [ ]:
outliers_z["ventas"].describe()

Comparación rápida

In [ ]:
print("IQR outliers (precio_unitario):", out_iqr_precio.shape[0])
print("Z-score outliers (ventas):", outliers_z.shape[0])

<p>IQR es más robusto; Z-score es más estadístico. Ninguno es mejor en abstracto: depende del problema.”</p>

<h5 style="color:#1f77b4;">7. Visualizar outliers con boxplots y scatter plots.</h5>

Boxplot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 3))
sns.boxplot(x=df_demo["precio_unitario"])
plt.title("Boxplot de precio_unitario")
plt.show()


In [ ]:
plt.figure(figsize=(6, 3))
sns.boxplot(x=df_demo["ventas"])
plt.title("Boxplot de ventas")
plt.show()


Scatter plot

Scatter simple por índice

In [ ]:
plt.figure(figsize=(7, 3))
plt.scatter(range(len(df_demo)), df_demo["ventas"])
plt.title("Scatter de ventas por índice")
plt.xlabel("Índice (fila)")
plt.ylabel("Ventas")
plt.show()


Scatter entre dos variables

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(df_demo["precio_unitario"], df_demo["ventas"])
plt.title("Precio unitario vs Ventas")
plt.xlabel("Precio unitario")
plt.ylabel("Ventas")
plt.show()
